In [1]:
import os

In [2]:
from keras.preprocessing import image

2022-12-13 23:49:57.752795: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-13 23:49:57.753033: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-13 23:50:09.791647: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/api/keras/optimizers


AlreadyExistsError: Another metric with the same name already exists.

In [4]:
import cv2

In [3]:
categories = ['with_mask','without_mask']

In [9]:
data = []
for category in categories:
  path = os.path.join('train',category)
  # print(path)

  label = categories.index(category)

  for file in os.listdir(path):

    img_path = os.path.join(path,file)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224,224))

    data.append([img,label])
    

In [10]:
len(data)

1006

In [11]:
import random

In [12]:
random.shuffle(data)

In [13]:
X = []
y = []

for features,label in data:
  X.append(features)
  y.append(label)

In [14]:
len(X)

1006

In [15]:
len(y)

1006

In [16]:
import numpy as np

In [17]:
X = np.array(X)
y = np.array(y)

In [18]:
X.shape

(1006, 224, 224, 3)

In [19]:
y.shape

(1006,)

In [20]:
X = X/255

In [22]:
pip install scikit-learn

     |████████████████████████████████| 9.7 MB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 297 kB 2.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [24]:
X_train.shape

(804, 224, 224, 3)

In [25]:
X_test.shape

(202, 224, 224, 3)

In [32]:
import tensorflow as tf

In [35]:
from tensorflow.keras.applications.vgg16 import VGG16

In [36]:
vgg = VGG16()

2022-12-14 00:04:32.993100: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/siddharth/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-12-14 00:04:33.023344: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-14 00:04:33.032253: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (siddharth): /proc/driver/nvidia/version does not exist
2022-12-14 00:04:33.130823: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-14 00:04:36.095892:

553467904/553467096 [==============================] - 126s 0us/step


In [37]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [39]:
from tensorflow.keras import Sequential

In [40]:
model = Sequential()

In [41]:
for layer in vgg.layers[:-1]:
  model.add(layer)

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [43]:
for layer in model.layers:
  layer.trainable = False

In [44]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [46]:
from tensorflow.keras.layers import Dense

In [47]:
model.add(Dense(1,activation='sigmoid'))

In [48]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [49]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [50]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

2022-12-14 00:13:39.514762: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-12-14 00:13:40.181154: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2799925000 Hz


Epoch 1/5
26/26 [==============================] - 179s 6s/step - loss: 0.5963 - accuracy: 0.7301 - val_loss: 0.4678 - val_accuracy: 0.8861
Epoch 2/5
26/26 [==============================] - 135s 5s/step - loss: 0.3894 - accuracy: 0.8930 - val_loss: 0.3733 - val_accuracy: 0.9010
Epoch 3/5
26/26 [==============================] - 140s 5s/step - loss: 0.3036 - accuracy: 0.9167 - val_loss: 0.3471 - val_accuracy: 0.9010
Epoch 4/5
26/26 [==============================] - 144s 6s/step - loss: 0.2527 - accuracy: 0.9353 - val_loss: 0.2914 - val_accuracy: 0.9109
Epoch 5/5
26/26 [==============================] - 149s 6s/step - loss: 0.2281 - accuracy: 0.9403 - val_loss: 0.2758 - val_accuracy: 0.9158


In [51]:
model.save('mask.model',save_format="h5")

In [7]:
# import pickle
# import joblib

# with open('model_pkl','wb') as files:
#     pickle.dump(model,files)

# joblib.dump(model,'model')
# pickle.dump(model,open('model.pkl','wb'))

In [1]:
from tensorflow.keras.models import load_model
import cv2

2022-12-14 15:25:15.383157: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-14 15:25:15.383252: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
model = load_model("mask.model")

2022-12-14 15:25:34.269145: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/siddharth/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-12-14 15:25:34.269253: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-14 15:25:34.269370: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (siddharth): /proc/driver/nvidia/version does not exist
2022-12-14 15:25:34.318778: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-14 15:25:35.208343:

In [11]:
cap = cv2.VideoCapture('0')

In [12]:
def detect_face_mask(img):
  y_pred = model.predict_classes(img.reshape(1,224,224,3))
  return y_pred[0][0]

In [13]:
def draw_label(img, text, pos,bg_color):

  text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)

  end_x = pos[0] + text_size[0][0] + 2
  end_y = pos[1] + text_size[0][1] - 2

  cv2.rectangle(img, pos, (end_x, end_y), bg_color, cv2.FILLED)
  cv2.putText(img,text,pos,cv2. FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

In [14]:
while True:

  ret, frame = cap.read()

  img = cv2.resize(frame,(224,224))
  y_pred = detect_face_mask(img)

  coods = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))

  for x,y,w,h in coods:
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)

  if y_pred == 0:
    draw_label(frame,"Mask",(30,30),(0,255,0))
  else:
    draw_label(frame,"No Mask",(30,30),(0,0,255))

  cv2.imshow('window',frame)
  if cv2.waitKey(1) & 0xFF == ord('x'):
    break
  
cv2.destroyAllWindows()

/home/siddharth/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [30]:
# DUMMY

while True:

  ret, frame = cap.read()

  img = cv2.resize(frame,(224,224))
  y_pred = detect_face_mask(img)

  coods = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))

  for x,y,w,h in coods:
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)

    if y_pred == 0:
        # draw_label(frame,"Mask",(30,30),(0,255,0))
        cv2.putText(frame,"Mask",(x,y-5),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,255,0),1)
    else:
        # draw_label(frame,"No Mask",(30,30),(0,0,255))
        cv2.putText(frame,"No Mask",(x,y-5),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,255),1)

  cv2.imshow('window',frame)
  if cv2.waitKey(1) & 0xFF == ord('x'):
    break
  
cv2.destroyAllWindows()

/home/siddharth/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [24]:
##################################

In [9]:
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [7]:
def detect_face(img):
    coods = haar.detectMultiScale(img)
    return coods

In [27]:
while True:

    ret,frame = cap.read()

    coods = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))

    for x,y,w,h in coods:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)

    cv2.imshow('window',frame)

    if cv2.waitKey(1) & 0xFF == ord('x'):
        break

cv2.destroyAllWindows()